In [1]:
import os
import cv2
import random
import numpy as np
from glob import glob
from pathlib import Path

In [2]:
# === Funciones ===
def random_background(size):
    base_color = np.random.randint(200, 256, size=3, dtype=np.uint8)  # aseguramos tipo
    bg = np.ones((size, size, 3), dtype=np.uint8) * base_color
    noise = np.random.randint(0, 30, (size, size, 3), dtype=np.uint8)
    return cv2.add(bg, noise.astype(np.uint8))  # forzamos uint8 por si acaso


def apply_augmentation(card_img):
    scale = random.uniform(*card_scale_range)
    angle = random.uniform(*rotation_range)
    h, w = card_img.shape[:2]
    new_w, new_h = int(w * scale), int(h * scale)
    card_resized = cv2.resize(card_img, (new_w, new_h))
    M = cv2.getRotationMatrix2D((new_w // 2, new_h // 2), angle, 1.0)
    rotated = cv2.warpAffine(card_resized, M, (new_w, new_h), borderValue=(255,255,255))
    if random.random() < blur_prob:
        rotated = cv2.GaussianBlur(rotated, (5, 5), 0)
    return rotated

def paste_card(bg, card, y_zone):
    ch, cw = card.shape[:2]
    x = random.randint(0, img_size - cw)
    y_range = (0, img_size // 2 - ch) if y_zone == "DEALER" else (img_size // 2, img_size - ch)
    y = random.randint(*y_range)
    region = bg[y:y+ch, x:x+cw]
    mask = (card < 250).any(axis=2)
    region[mask] = card[mask]
    return bg, (x, y, x + cw, y + ch)

def save_yolo_label(path, box, class_id):
    x1, y1, x2, y2 = box
    cx = (x1 + x2) / 2 / img_size
    cy = (y1 + y2) / 2 / img_size
    w = (x2 - x1) / img_size
    h = (y2 - y1) / img_size
    with open(path, "w") as f:
        f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")


In [3]:
# === Configuracion ===
input_root = "poker/train"                     # Origen: cartas individuales
output_img_dir = "cards/images/train"          # Destino: imagenes YOLO
output_lbl_dir = "cards/labels/train"          # Destino: labels YOLO
img_size = 960
images_per_card = 3                           # Generar 10 por cada original
card_scale_range = (0.4, 0.6)                  # Tamaño relativo
rotation_range = (-10, 10)                     # Rotacion leve
blur_prob = 0.3                                # Probabilidad de aplicar desenfoque

In [4]:
# Crear carpetas de salida si no existen
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

In [5]:
# Obtener clases
clases = sorted(os.listdir(input_root))
class_to_index = {name: i for i, name in enumerate(clases)}

In [6]:
# === Generar imagenes con progreso ===
count = 0
total_clases = len(clases)

for class_idx, class_name in enumerate(clases):
    class_path = os.path.join(input_root, class_name)
    img_paths = glob(os.path.join(class_path, "*.jpg"))
    class_id = class_to_index[class_name]

    for img_idx, img_path in enumerate(img_paths):
        original = cv2.imread(img_path)
        if original is None:
            continue

        for i in range(images_per_card):
            # Mostrar progreso
            print(f"\r🛠️ {count+1:5d} | Clase {class_idx+1}/{total_clases} → '{class_name}' "
                  f"[imagen {img_idx+1}/{len(img_paths)} | aug {i+1}/{images_per_card}]", end='', flush=True)

            # Proceso de augmentación
            bg = random_background(img_size)
            card_aug = apply_augmentation(original)
            y_zone = "DEALER" if i < images_per_card // 2 else "JUGADOR"
            bg_final, bbox = paste_card(bg.copy(), card_aug, y_zone)

            fname = f"aug_{class_name.replace(' ', '_')}_{count}.jpg"
            img_out = os.path.join(output_img_dir, fname)
            lbl_out = os.path.join(output_lbl_dir, fname.replace(".jpg", ".txt"))

            cv2.imwrite(img_out, bg_final)
            save_yolo_label(lbl_out, bbox, class_id)
            count += 1

print(f"\n✅ Generadas {count} imágenes augmentadas en formato YOLOv8.")


🛠️     1 | Clase 1/53 → 'ace of clubs' [imagen 1/120 | aug 1/3]

🛠️ 22872 | Clase 53/53 → 'two of spades' [imagen 155/155 | aug 3/3]3]3]
✅ Generadas 22872 imágenes augmentadas en formato YOLOv8.
